In [1]:
!python license-plate-detection.py ../ETC_data/fail_etc models/etc_models/model_v2_12600/my-trained-model_backupit_12600

Traceback (most recent call last):
  File "license-plate-detection.py", line 4, in <module>
    import keras
ModuleNotFoundError: No module named 'keras'


In [2]:
import sys, os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import keras
import cv2
import traceback
import numpy as np 

from glob import glob
from os.path import splitext, basename

from src.utils import im2single
from src.keras_utils2 import load_model, detect_lp,detect_lp_on_batch
# from src.keras_utils import load_model, detect_lp
from src.label import Shape, writeShapes
import time
import matplotlib.pylot as plt

def adjust_pts(pts, lroi):
    return pts * lroi.wh().reshape((2, 1)) + lroi.tl().reshape((2, 1))



def alpr_unconstrained_detector_predict(wpod_net,Ivehicle):
    lp_threshold = 0.5
    ratio = float(max(Ivehicle.shape[:2])) / min(Ivehicle.shape[:2])
    side = int(ratio * 288.)
    bound_dim = min(side + (side % (2 ** 4)), 608)
    for i in range (10):
        print("input shape:", np.shape(Ivehicle))
        start = time.time()  
        
        Llp, LlpImgs, elapsed = detect_lp_on_batch(wpod_net, [im2single(Ivehicle[Ivehicle.shape[0]//2:,:,:])]*16, bound_dim, 2 ** 4, (240, 80), lp_threshold)
    #     Llp, LlpImgs, elapsed = detect_lp(wpod_net, im2single(Ivehicle), bound_dim, 2 ** 4, (240, 80), lp_threshold)
        end = time.time()
    #     print("Time network forward: ", elapsed)
        print("Time detect: ", end -start)
#     return None  ,(end-start)
    LlpImgs = LlpImgs[0]
#     Llp = Llp[0]
    for label in Llp:
        x0, x1, x2, x3, y0, y1, y2, y3 = np.reshape(label.pts,-1)  
        pred_pts = [(x0, y0), (x1, y1), (x2, y2), (x3, y3)]
        h,w = np.shape(Ivehicle)[:2]
        pts2 = [(x0*w, y0*h), (x1*w, y1*h), (x2*w, y2*h), (x3*w, y3*h)]  
        pts_arr = np.asarray([pts2[0],pts2[3],pts2[2],pts2[1]],np.int32)
        img=cv2.polylines(Ivehicle,[pts_arr],True,(0,255,255),2)
        plt.imshow(Ivehicle)
        plt.show()
    if len(LlpImgs):
        Ilp = LlpImgs[0]
        Ilp = cv2.cvtColor(Ilp, cv2.COLOR_BGR2GRAY)
        Ilp = cv2.cvtColor(Ilp, cv2.COLOR_GRAY2BGR)
        Ilp =   Ilp * 255. 
        return Ilp,(end-start)
    else:
        return None  ,(end-start)

# class alpr_unconstrained_detector():
#     def __init__(self,wpod_net_path=""):
#         self.wpod_net_path = wpod_net_path
#         self.wpod_net = load_model(self.wpod_net_path)
#     def predict(self,Ivehicle):
#         lp_threshold = 0.5
#         ratio = float(max(Ivehicle.shape[:2])) / min(Ivehicle.shape[:2])
#         side = int(ratio * 288.)
#         bound_dim = min(side + (side % (2 ** 4)), 608)
#         print("input shape:", np.shape(Ivehicle))
#         print ("\t\tBound dim: %d, ratio: %f" % (bound_dim, ratio))
#         start = time.time()                
#         Llp, LlpImgs, elapsed = detect_lp(self.wpod_net, im2single(Ivehicle), bound_dim, 2 ** 4, (240, 80), lp_threshold)
#         end = time.time()
#         print("Time network forward: ", elapsed)
#         print("Time detect: ", end -start)
#         if len(LlpImgs):
#             Ilp = LlpImgs[0]
#             Ilp = cv2.cvtColor(Ilp, cv2.COLOR_BGR2GRAY)
#             Ilp = cv2.cvtColor(Ilp, cv2.COLOR_GRAY2BGR)
#             Llp =   Ilp * 255. 
#             return Llp
#         else:
#             return None  



if __name__ == '__main__':

    try:

        input_dir = sys.argv[1] 
        output_dir = "./output"
        lp_threshold = 0.5

        wpod_net_path = sys.argv[2]
        wpod_net = load_model(wpod_net_path)
        total_time = 0
        start = time.time() 
                        
#         image_paths = glob('%s/*.jpg' % input_dir)
        image_paths = glob('%s/*.png' % input_dir)
        
        for root,dirr,files in os.walk(input_dir):
            for item in files:
                if ('.jpg' in item) or ('.png' in item):
                    file = root+'/'+item
                    image_paths.append(file)
                    
        
        # 		imgs_paths = glob('%s/*.png' % input_dir)

        print('Searching for license plates using WPOD-NET')

        print("number of images:", len(image_paths))
        for i, img_path in enumerate(image_paths):
            out_dir = output_dir+"/"+img_path.replace("../","").replace("jpg","png") 
            dir_name = os.path.dirname(out_dir)
#             if os.path.exists(out_dir) == True:
#                 continue
            
            bname = splitext(basename(img_path))[0] 
            print ('\t Processing %s' % img_path)
            Ivehicle = cv2.imread(img_path)   # load gray
            Ilp,timep=alpr_unconstrained_detector_predict(wpod_net,Ivehicle)
            if i>=0:
                total_time+=timep 

            if (Ilp is None)==False: 
                
                if os.path.exists(dir_name) == False:
                    os.makedirs(dir_name)
                cv2.imwrite(out_dir, Ilp )  
        print("mean_time:",time.time()-start)
    
    except:
        traceback.print_exc()
        sys.exit(1)
    sys.exit(0)

# python license-plate-detection.py etc_raw lp-detector/wpod-net_update1
 
# python license-plate-detection.py ./data/Image models/etc/my-trained-model_backupit_12600
# python license-plate-detection.py ../../../etc_data models/etc/model_v2_12600/my-trained-model_backupit_12600
#python license-plate-detection.py ./val/Viet_VVT_POC_image_20200204/dataset3 models/etc/my-trained-model_backupit_12600

#python license-plate-detection.py ../license-plate-detector-data/val/Viet_VVT_POC_image_20200204/dataset3 models/etc_models/model_v2_12600/my-trained-model_backupit_12600

# moved all data to alpr-unconstrained-master-data



#python license-plate-detection.py ../license-plate-detector-data/val models/etc_models/model_v2_12600/my-trained-model_backupit_12600

#python license-plate-detection.py ../ETC_data/fail_etc models/etc_models/model_v2_12600/my-trained-model_backupit_12600

Using TensorFlow backend.
/home/utvm/anaconda3/envs/anhnt/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/utvm/anaconda3/envs/anhnt/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/utvm/anaconda3/envs/anhnt/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/utvm/anaconda3/envs/anh